In [19]:
# import dependency
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import LlamaCpp

In [20]:
# craft a prompt template that works best for your LLM
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up answer.
{context}
Question: {question}
Answer:"""

In [21]:
# context will be similar doc and question will be query
prompt = PromptTemplate.from_template(template)
prompt.input_variables

['context', 'question']

### create vector space and embeddings

In [22]:
# import dependency
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

# load text file
loader = TextLoader("../temp/file.txt")
docs = loader.load()

# transform into chunks
text_splitter = CharacterTextSplitter(chunk_size=10, chunk_overlap=0)
texts = text_splitter.split_documents(docs)

# import dependency
from langchain.embeddings import LlamaCppEmbeddings
embeddings = LlamaCppEmbeddings(model_path="../models/llama-7b.ggmlv3.q5_K_S.bin")

# convert langchain doc to str
_texts = []
for i in range(len(texts)):
    _texts.append(texts[i].page_content)
    
# embedded list of string
embedded_text = embeddings.embed_documents(_texts)

# import dependency
from langchain.vectorstores import Chroma

# Create a chrom vector store from a list of documents
db = Chroma.from_documents(texts, embeddings)

Created a chunk of size 298, which is longer than the specified 10
Created a chunk of size 218, which is longer than the specified 10


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


In [23]:
db.get()

{'ids': ['a3118ebe-48aa-11ee-b0ad-f0921cf646bb',
  'a3118ebf-48aa-11ee-9b57-f0921cf646bb',
  'a3118ec0-48aa-11ee-b708-f0921cf646bb'],
 'embeddings': None,
 'documents': ['As a baby, his parents sent him to Earth in a small spaceship moments before Krypton was destroyed in a natural cataclysm. His ship landed in the American countryside, near the fictional town of Smallville. He was found and adopted by farmers Jonathan and Martha Kent, who named him Clark Kent.',
  'In her homeland, the island nation of Themyscira, her official title is Princess Diana of Themyscira. When blending into the society outside of her homeland, she sometimes adopts her civilian identity Diana Prince.',
  'This new Flash was Barry Allen, a police scientist who gained super-speed when bathed by chemicals after a shelf of them was struck by lightning. He adopted the name The Scarlet Speedster after reading a comic book featuring the Golden Age Flash.'],
 'metadatas': [{'source': '../temp/file.txt'},
  {'source':

In [24]:
# import llm
llm = LlamaCpp(model_path="../models/llama-7b.ggmlv3.q5_K_S.bin")

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


In [25]:
# prepare context from query
query_question = "his parents send him?"

similarity_doc = db.similarity_search(query_question, k=1)
context = similarity_doc[0].page_content
print(context)

In her homeland, the island nation of Themyscira, her official title is Princess Diana of Themyscira. When blending into the society outside of her homeland, she sometimes adopts her civilian identity Diana Prince.


In [26]:
similarity_doc

[Document(page_content='In her homeland, the island nation of Themyscira, her official title is Princess Diana of Themyscira. When blending into the society outside of her homeland, she sometimes adopts her civilian identity Diana Prince.', metadata={'source': '../temp/file.txt'})]

In [29]:
query_llm = LLMChain(llm=llm, prompt=prompt)
response = query_llm({ "context": context, "question": query_llm})
print(response)

Llama.generate: prefix-match hit


In [ ]:
response['text']

" ___________\nQuestion: The following information was taken from the cover of ________ #1 (2014) and an issue of Justice League Vol 5 (2018).\nAnswer: __________________\nQuestion: What does 'the Worlds Finest' refer to?\nAnswer: ___________\nQuestion: what are these two things?\nAnswer: ____________"